In [7]:
WINS = [(0,1,2),(3,4,5),(6,7,8),(0,3,6),(1,4,7),(2,5,8),(0,4,8),(2,4,6)]

def winner(s):
    for a, b, c in WINS:
        if s[a] == s[b] == s[c] != 0:
            return s[a]
    return None

def is_terminal(s):
    return winner(s) is not None or 0 not in s

def turn(s):
    return 1 if s.count(1) == s.count(-1) else -1

def successors(s):
    p = turn(s)
    return [(i, s[:i] + (p,) + s[i+1:]) for i in range(9) if s[i] == 0]

In [8]:
def value_iteration(theta=1e-6):
    visited, queue = {(0,)*9}, [(0,)*9]
    while queue:
        s = queue.pop()
        if not is_terminal(s):
            for _, s2 in successors(s):
                if s2 not in visited:
                    visited.add(s2)
                    queue.append(s2)

    V = {s: float(winner(s)) if winner(s) else 0.0 for s in visited}

    for iteration in range(1, 1000):
        delta = 0.0
        for s in visited:
            if is_terminal(s):
                continue
            old = V[s]
            child_values = [V[s2] for _, s2 in successors(s)]
            V[s] = max(child_values) if turn(s) == 1 else min(child_values)
            delta = max(delta, abs(V[s] - old))

        print(f"Iteration {iteration:3d}  |  max delta = {delta:.2e}")
        if delta < theta:
            break

    return V


def best_move(s, V):
    p = turn(s)
    moves = successors(s)
    return max(moves, key=lambda x: V[x[1]])[0] if p == 1 \
      else min(moves, key=lambda x: V[x[1]])[0]


print("Running value iteration...\n")
V = value_iteration()

print(f"\nConverged. {len(V)} states in table.")
print(f"Value of empty board (expect 0.0 = draw): {V[(0,)*9]}")

s = (0,) * 9
while not is_terminal(s):
    m = best_move(s, V)
    s = s[:m] + (turn(s),) + s[m+1:]
print(f"Optimal self-play result: { {1:'X wins',-1:'O wins',None:'Draw'}[winner(s)] }")

Running value iteration...

Iteration   1  |  max delta = 1.00e+00
Iteration   2  |  max delta = 1.00e+00
Iteration   3  |  max delta = 1.00e+00
Iteration   4  |  max delta = 1.00e+00
Iteration   5  |  max delta = 0.00e+00

Converged. 5478 states in table.
Value of empty board (expect 0.0 = draw): 0.0
Optimal self-play result: Draw


In [ ]:
def policy_iteration(theta=1e-6):
    visited, queue = {(0,)*9}, [(0,)*9]
    while queue:
        s = queue.pop()
        if not is_terminal(s):
            for _, s2 in successors(s):
                if s2 not in visited:
                    visited.add(s2)
                    queue.append(s2)

    non_terminal = [s for s in visited if not is_terminal(s)]

    V = {s: float(winner(s)) if winner(s) else 0.0 for s in visited}
    policy = {s: successors(s)[0][0] for s in non_terminal}

    for iteration in range(1, 1000):
        while True:
            delta = 0.0
            for s in non_terminal:
                old = V[s]
                m = policy[s]
                p = turn(s)
                s_next = s[:m] + (p,) + s[m+1:]
                V[s] = V[s_next]
                delta = max(delta, abs(V[s] - old))
            if delta < theta:
                break

        policy_stable = True
        improvements = 0
        for s in non_terminal:
            old_action = policy[s]
            child_values = [(i, V[s2]) for i, s2 in successors(s)]
            if turn(s) == 1:
                best = max(child_values, key=lambda x: x[1])[0]
            else:
                best = min(child_values, key=lambda x: x[1])[0]

            if best != old_action:
                policy[s] = best
                policy_stable = False
                improvements += 1

        print(f"Iteration {iteration}  |  policy changes = {improvements}")
        if policy_stable:
            break

    return V, policy


def best_move(s, policy):
    return policy[s]


print("Running policy iteration...\n")
V, policy = policy_iteration()

print(f"\nConverged. {len(V)} states in table.")
print(f"Value of empty board (expect 0.0 = draw): {V[(0,)*9]}")

s = (0,) * 9
while not is_terminal(s):
    m = best_move(s, policy)
    s = s[:m] + (turn(s),) + s[m+1:]
print(f"Optimal self-play result: { {1:'X wins',-1:'O wins',None:'Draw'}[winner(s)] }")

Running policy iteration...

Iteration 1  |  policy changes = 1567
Iteration 2  |  policy changes = 1532
Iteration 3  |  policy changes = 830
Iteration 4  |  policy changes = 358
Iteration 5  |  policy changes = 135
Iteration 6  |  policy changes = 28
Iteration 7  |  policy changes = 6
Iteration 8  |  policy changes = 1
Iteration 9  |  policy changes = 0

Converged. 5478 states in table.
Value of empty board (expect 0.0 = draw): 0.0
Optimal self-play result: Draw
